### Getting Started

## Where do ensembles come from?

In the constraint-based reconstruction and analysis (COBRA) field, there are a variety of analyses and approaches that have alternative optimal solutions. Examples of this come up often when analyses involve [integer programming](https://en.wikipedia.org/wiki/Integer_programming), where variables in a problem are switched on/off to maximize or minimize an objective value. When integer programming is applied to fill in gaps in a genome-scale metabolic model (GEM) to fulfill a function, such as production of biomass, there are generally multiple sets of equally likely solutions to the problem (e.g. there is more than one unique set of reactions of the same size that enable growth).

Beyond examples of analyses where multiple solutions with the *exact same* objective value might exist, keeping track of multiple sub-optimal solutions might be valuable because a biologically-correct solution is almost never the exact same as the most likely numerical solution. Instead, we can hedge our bets by maintaining an ensemble of feasible solutions to a problem that all meet some minimum acceptable likelihood.

The primary aim of medusa is to make ensemble analyses more accessible to the COBRA field so that we can start accounting for this uncertainty to improve our predictions and improve our reconstructions more efficiently. The software is written and developed with usability as the top priority, secondary to performance and novelty. Thus, user feedback is essential--please do not hesitate to provide your thoughts or ask questions via an issue on the [medusa github repository](https://github.com/gregmedlock/Medusa/).


## Loading a model and inspecting it

The easiest way to see the contents of ensembles in Medusa is to load test models. Here, we use a function that takes the E. coli core metabolism reconstruction from cobrapy and randomly removes components to generate ensemble members.


In [1]:
import medusa
from medusa.test.test_ensemble import construct_textbook_ensemble

example_ensemble = construct_textbook_ensemble()

In medusa, we represent an ensemble of genome-scale metabolic network reconstructions (GENREs) using the `medusa.Ensemble` object. Each `Ensemble` has three key attributes that specify the structure of the ensemble.

## Components of an ensemble: base_model

The first is the `base_model`, which is a `cobra.Model` object that represents all the possible states of an individual member within the ensemble. Any reaction, metabolite, or gene that is only present in a subset of ensemble members will be present in `base_model`. You can inspect the `base_model` and manipulate it just like any other `cobra.Model` object.

In [2]:
extracted_base_model = example_ensemble.base_model
extracted_base_model

Name,first_textbook
Memory address,0x07fe3c110df98
Number of metabolites,72
Number of reactions,95
Objective expression,1.0*Biomass_Ecoli_core - 1.0*Biomass_Ecoli_core_reverse_2cdba
Compartments,"extracellular, cytosol"


## Components of an ensemble: members

The second attribute that each `Ensemble` has is a structure called `members`. `Ensemble.members` maps an identifier for each individual GENRE in the ensemble to a `medusa.Member` object, which holds information about a single member. 

`Ensemble.members` is represented by a custom datatype implemented in cobrapy called a DictList, which is essentially a standard dictionary in python that can also be accessed using integer indices like a list (e.g. dictlist[0] returns the first element in the dictlist).

In [3]:
# looks like a list when we print it
example_ensemble.members

[<Member first_textbook at 0x7fe3c1286e80>,
 <Member second_textbook at 0x7fe3c1286358>]

In [4]:
# Get the first member with integer indexing
first_member = example_ensemble.members[0]

Each `Member` within the `Ensemble.members` DictList has a handful of attributes as well. You can check the ensemble that the member belongs to, the id of the member, and the network states for that member (we'll discuss states more below).

In [5]:
print(first_member.ensemble)
print(first_member.id)
print(first_member.states)

textbook_ensemble
first_textbook
{<Feature ACt2r_upper_bound at 0x7fe3c116db38>: 1000.0, <Feature ACt2r_lower_bound at 0x7fe3c116da58>: -1000.0, <Feature ACALDt_lower_bound at 0x7fe3c116d550>: 0.0, <Feature ACALDt_upper_bound at 0x7fe3c116d278>: 0.0, <Feature ACKr_upper_bound at 0x7fe3c116d3c8>: 0.0, <Feature ACONTb_lower_bound at 0x7fe3c116d9b0>: -1000.0, <Feature ACONTb_upper_bound at 0x7fe3c116dac8>: 1000.0, <Feature ACKr_lower_bound at 0x7fe3c116d5f8>: 0.0}


## Components of an ensemble: features

The states printed above are directly connected to the third attribute that `Ensemble` contains, `Ensemble.features`, which is also a DictList object. `Ensemble.features` contains `medusa.Feature` entries, which specify the components of the `Ensemble.base_model` that vary across the entire ensemble. 

In [6]:
example_ensemble.features

[<Feature ACONTb_lower_bound at 0x7fe3c116d9b0>,
 <Feature ACONTb_upper_bound at 0x7fe3c116dac8>,
 <Feature ACt2r_lower_bound at 0x7fe3c116da58>,
 <Feature ACt2r_upper_bound at 0x7fe3c116db38>,
 <Feature ACKr_lower_bound at 0x7fe3c116d5f8>,
 <Feature ACKr_upper_bound at 0x7fe3c116d3c8>,
 <Feature ACALDt_lower_bound at 0x7fe3c116d550>,
 <Feature ACALDt_upper_bound at 0x7fe3c116d278>]

Here, we see that this `Ensemble` has 8 features. Each `Feature` object specifies a network component that has a variable parameter value in at least one member of the ensemble (e.g. at least one ensemble member is missing the reaction).

In this case, there are features for 4 reactions, `ACALDt`,`ACKr`,`ACONTb`, and `ACt2r`. There are two `Feature` objects for each reaction, corresponding to the lower and upper bound for that reaction.

This information can be inferred from feature ID (`medusa.feature.id`), but each `Feature` also has a set of attributes that encode the information. Some useful attributes, in order the order printed below:  getting the `Ensemble` that the `Feature` belongs to, the component in the `Ensemble.base_model` that the `Feature` describes, the attribute of the component in the `Ensemble.base_model` whose value the `Feature` specifies, and the ID of the `Feature`:

In [7]:
first_feature = example_ensemble.features[0]
print(first_feature.ensemble)
print(first_feature.base_component)
print(first_feature.component_attribute)
print(first_feature.id)

textbook_ensemble
ACONTb: acon_C_c + h2o_c <=> icit_c
lower_bound
ACONTb_lower_bound


Just as each `member` has an attribute, `states`, that returns the value of every feature for that `member`, each `feature` has a `states` dictionary that maps each `member.id` to the value of the `feature` in the corresponding `member`, e.g.:

In [8]:
print(first_feature.states)

{'first_textbook': -1000.0, 'second_textbook': 0.0}


## Strategies for getting information about an ensemble and its members

Where possible, we recycled conventions from cobrapy for accessing information about attributes. In cobrapy, the `Model` object has multiple containers in the form of DictLists: `Model.reactions`,`Model.metabolites`,`Model.genes`. Equivalently in medusa, each `Ensemble` has similarly constructed containers: `Ensemble.members` and `Ensemble.features`.

As such, information about specific `Member` and `Feature` objects can be accessed just like `Reaction`, `Metabolite`, and `Gene` objects in cobrapy:

In [9]:
# remember, our Ensemble holds a normal cobrapy Model in base_model
extracted_base_model = example_ensemble.base_model
# Accessing object by id is common in cobrapy
rxn = extracted_base_model.reactions.get_by_id('ACALDt')
# We can do the same thing for features:
feat = example_ensemble.features.get_by_id('ACALDt_lower_bound')
print(rxn)
print(feat.base_component)
print(feat.component_attribute)

# And for members:
memb = example_ensemble.members.get_by_id('first_textbook')
print('\nHere are the states for this member:')
print(memb.states)

ACALDt: acald_e <=> acald_c
ACALDt: acald_e <=> acald_c
lower_bound

Here are the states for this member:
{<Feature ACt2r_upper_bound at 0x7fe3c116db38>: 1000.0, <Feature ACt2r_lower_bound at 0x7fe3c116da58>: -1000.0, <Feature ACALDt_lower_bound at 0x7fe3c116d550>: 0.0, <Feature ACALDt_upper_bound at 0x7fe3c116d278>: 0.0, <Feature ACKr_upper_bound at 0x7fe3c116d3c8>: 0.0, <Feature ACONTb_lower_bound at 0x7fe3c116d9b0>: -1000.0, <Feature ACONTb_upper_bound at 0x7fe3c116dac8>: 1000.0, <Feature ACKr_lower_bound at 0x7fe3c116d5f8>: 0.0}


These DictList objects are all [iterables](https://docs.python.org/3/glossary.html#term-iterable), meaning that any python operation that acts on an iterable can take them as input. This is often convenient when working with either cobrapy `Model`s or medusa `Ensemble`s. For example, suppose we are interested in getting the list of all components described by features in the `Ensemble`:

In [10]:
components = []
for feat in example_ensemble.features:
    components.append(feat.base_component)

print(components)

# or, use the one-liner which gives the same result:
components = [feat.base_component for feat in example_ensemble.features]

print(components)

[<Reaction ACONTb at 0x7fe3c1035470>, <Reaction ACONTb at 0x7fe3c1035470>, <Reaction ACt2r at 0x7fe3c1035c88>, <Reaction ACt2r at 0x7fe3c1035c88>, <Reaction ACKr at 0x7fe3c0f6f978>, <Reaction ACKr at 0x7fe3c0f6f978>, <Reaction ACALDt at 0x7fe3c0f6f908>, <Reaction ACALDt at 0x7fe3c0f6f908>]
[<Reaction ACONTb at 0x7fe3c1035470>, <Reaction ACONTb at 0x7fe3c1035470>, <Reaction ACt2r at 0x7fe3c1035c88>, <Reaction ACt2r at 0x7fe3c1035c88>, <Reaction ACKr at 0x7fe3c0f6f978>, <Reaction ACKr at 0x7fe3c0f6f978>, <Reaction ACALDt at 0x7fe3c0f6f908>, <Reaction ACALDt at 0x7fe3c0f6f908>]
